In [1]:
import pandas as pd

In [2]:
'''
Parámetros de entrada: dataframe y dos columnas de tiempo
Salida: Cinco nuevas listas obtenidas a partir de dos columnas de tiempo el dataframe de entrada
'''

def transformacion_tiempo(df, *columnas):
    
    #Importación de paqueterías auxiliares en la manipulación de fechas
    from datetime import datetime, timedelta, time
    import calendar as cl
    
    #Columnas por crear para insertar
    start_day = list()
    start_hour = list()
    end_day = list()
    end_hour = list()
    trip_duration = list()
    
    for ind in df.index:

        marca_t_ini = df[columnas[0]].loc[ind]
        marca_t_fin = df[columnas[1]].loc[ind]
        
        try:
            fecha_h_ini = datetime.strptime(marca_t_ini.split('.')[0], '%Y-%m-%d %H:%M:%S')
        except:
            fecha_h_ini = datetime.strptime(marca_t_ini, '%Y-%m-%d %H:%M:%S')
        week_day_ini = fecha_h_ini.weekday()
        day_ini = cl.day_name[week_day_ini]

        hour_ini = fecha_h_ini.hour
        
        try:
            fecha_h_fin = datetime.strptime(marca_t_fin.split('.')[0], '%Y-%m-%d %H:%M:%S')
        except:
            fecha_h_fin = datetime.strptime(marca_t_fin, '%Y-%m-%d %H:%M:%S')
        week_day_fin = fecha_h_fin.weekday()
        day_fin = cl.day_name[week_day_fin]

        hour_fin = fecha_h_fin.hour

        tripduration = fecha_h_fin - fecha_h_ini
        minutes = round(tripduration.seconds/60,2)
        
        start_day.append(day_ini)
        start_hour.append(hour_ini)
        end_day.append(day_fin)
        end_hour.append(hour_fin)
        trip_duration.append(minutes)

    return start_day, start_hour, end_day, end_hour, trip_duration

# Transformación de los datos
La tabla de 2020 difiere en columnas con las de 2021 y 2022. El objetivo es hacer una sola tabla con las siguientes columnas, para después crear el modelo relacional:
1. ride_id
2. rideable_id {1: 'classic', 2: 'docked'}
3. *rideable_type* (classic, docked)
4. year
5. start_day_id {1:'monday', 2:'tuesday', 3:'wednesday', 4:'thursday', 5:'friday', 6:'saturday', 7:'sunday'}
6. *start_day* (monday, tuesday, wednesday, thursday, friday, saturday, sunday)
7. start_hour
8. end_day_id
9. *end_day*
10. end_hour
11. trip_duration: En minutos
12. start_station_id
13. *start_station_name*
14. *start_lat*
15. *start_lng*
16. end_station_id
17. *end_station_name*
18. *end_lat*
19. *end_lng*
20. member_id {1:'member', 2:'casual'}
21. *member_type* (annual, casual)
22. age (Obtenida del año de nacimiento para los registros de 2020)
23. gender

Las columnas en itálica conformarán nuevas tablas en el modelo relacional.
<br> Como se observa, se tendrán que transformar las columnas de marca de tiempo para obtener el año y el día de la semana. Al sólo haber un mes, podemos prescindir de él.

El procedimiento será similar para los tres años_
1. Renombrar columnas.
2. Crear nuevas columnas e insertarlas en la tabla.
3. Agregar columnas necesarias.
4. Eliminar columnas sobrantes.
5. Reordenar columnas.

## Transformación tabla 2020

In [3]:
jc_2020 = pd.read_csv('citibike//JC-202002-citibike-tripdata.csv')
[*jc_2020]

['tripduration',
 'starttime',
 'stoptime',
 'start station id',
 'start station name',
 'start station latitude',
 'start station longitude',
 'end station id',
 'end station name',
 'end station latitude',
 'end station longitude',
 'bikeid',
 'usertype',
 'birth year',
 'gender']

In [4]:
#Se renombran las columnas
jc_2020.rename(columns = {'start station id': 'start_station_id', 'start station name': 'start_station_name',
                          'start station latitude': 'start_lat', 'start station longitude': 'start_lng', 
                          'end station id': 'end_station_id', 'end station name': 'end_station_name',
                          'end station latitude': 'end_lat', 'end station longitude': 'end_lng', 
                          'usertype': 'member_type', 'birth year': 'birth_year'}, inplace = True)
[*jc_2020]

['tripduration',
 'starttime',
 'stoptime',
 'start_station_id',
 'start_station_name',
 'start_lat',
 'start_lng',
 'end_station_id',
 'end_station_name',
 'end_lat',
 'end_lng',
 'bikeid',
 'member_type',
 'birth_year',
 'gender']

In [5]:
#Nuevas columnas
lista_columnas = transformacion_tiempo(jc_2020, 'starttime', 'stoptime')

In [6]:
col = ['start_day', 'start_hour', 'end_day', 'end_hour', 'trip_duration']

i = 0
for c in range(3,8):
    jc_2020.insert(c, col[i], lista_columnas[i])
    i = i + 1
    
jc_2020

,tripduration,starttime,stoptime,start_day,start_hour,end_day,end_hour,trip_duration,start_station_id,start_station_name,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,bikeid,member_type,birth_year,gender
0,841,2020-02-01 00:02:13.7650,2020-02-01 00:16:15.4720,Saturday,0,Saturday,0,14.03,3199,Newport Pkwy,40.728745,-74.032108,3213,Van Vorst Park,40.718489,-74.047727,42217,Subscriber,1980,1
1,206,2020-02-01 00:16:11.3850,2020-02-01 00:19:37.4460,Saturday,0,Saturday,0,3.43,3273,Manila & 1st,40.721651,-74.042884,3639,Harborside,40.719252,-74.034234,42160,Subscriber,1990,1
2,859,2020-02-01 00:23:22.1090,2020-02-01 00:37:41.5060,Saturday,0,Saturday,0,14.32,3185,City Hall,40.717732,-74.043845,3268,Lafayette Park,40.713464,-74.062859,42204,Customer,1991,1
3,693,2020-02-01 00:25:14.5170,2020-02-01 00:36:48.0280,Saturday,0,Saturday,0,11.57,3185,City Hall,40.717732,-74.043845,3277,Communipaw & Berry Lane,40.714358,-74.066611,42165,Subscriber,1987,1
4,1473,2020-02-01 00:32:30.5050,2020-02-01 00:57:04.3740,Saturday,0,Saturday,0,24.57,3192,Liberty Light Rail,40.711242,-74.055701,3639,Harborside,40.719252,-74.034234,26156,Customer,1969,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22957,262,2020-02-29 23:40:53.1620,2020-02-29 23:45:15.3180,Saturday,23,Saturday,23,4.37,3639,Harborside,40.719252,-74.034234,3214,Essex Light Rail,40.712774,-74.036486,42467,Subscriber,1967,1
22958,216,2020-02-29 23:41:01.3350,2020-02-29 23:44:38.0350,Saturday,23,Saturday,23,3.62,3186,Grove St PATH,40.719586,-74.043117,3483,Montgomery St,40.719420,-74.050990,42377,Subscriber,1958,1
22959,101,2020-02-29 23:45:22.1700,2020-02-29 23:47:03.2970,Saturday,23,Saturday,23,1.68,3214,Essex Light Rail,40.712774,-74.036486,3681,Grand St,40.715178,-74.037683,42485,Subscriber,1967,1
22960,279,2020-02-29 23:47:56.8380,2020-02-29 23:52:36.6400,Saturday,23,Saturday,23,4.67,3186,Grove St PATH,40.719586,-74.043117,3483,Montgomery St,40.719420,-74.050990,42531,Subscriber,1988,1


In [7]:
#Columnas por rellenar.
jc_2020['ride_id'] = pd.Series()
jc_2020['rideable_id'] = pd.Series()
jc_2020['rideable_type'] = pd.Series()
jc_2020['member_id'] = pd.Series()
jc_2020['start_day_id'] = pd.Series()
jc_2020['end_day_id'] = pd.Series()
jc_2020['year'] = 2020

<ipython-input-7-56509db0c764>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  jc_2020['ride_id'] = pd.Series()
<ipython-input-7-56509db0c764>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  jc_2020['rideable_id'] = pd.Series()
<ipython-input-7-56509db0c764>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  jc_2020['rideable_type'] = pd.Series()
<ipython-input-7-56509db0c764>:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  jc_2020['member_id'] = pd.Series()
<ipython-input-7-56509db0c764>:6: DeprecationWar

In [8]:
#Rellenado de la columna 'ride_id'. Al desconocer la nomenclatura de las id que presenta en 2021 y 2022, se decide
# sólo hacer una id enumerada de 1 hasta la cantidad de elementos que sumen las tres tablas.

i = 0
for ind in jc_2020.index:
    i = i + 1
    jc_2020['ride_id'].loc[ind] = i
    
#Rellenado de la columna 'member_id' considerando la clasificación {1:'Subscriber', 2:'Customer'}
for ind in jc_2020.index:
    if jc_2020['member_type'].loc[ind] == 'Subscriber':
        jc_2020['member_id'].loc[ind] = 1
    else:
        jc_2020['member_id'].loc[ind] = 2
        
#Rellenado de la columna 'start_day_id' y 'end_day_id' considerando la clasificación 
# {1:'monday', 2:'tuesday', 3:'wednesday', 4:'thursday', 5:'friday', 6:'saturday', 7:'sunday'}
dias = {'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4, 'Friday':5, 'Saturday':6, 'Sunday':7}
for ind in jc_2020.index:
    day_start = jc_2020['start_day'].loc[ind]
    jc_2020['start_day_id'].loc[ind] = dias[day_start]
    day_end = jc_2020['end_day'].loc[ind]
    jc_2020['end_day_id'].loc[ind] = dias[day_end]
    
#Sobreescritura de la columna gender {0:'unspecified', 1:'male', 2:'female'}
for ind in jc_2020.index:
    gend = jc_2020['gender'].loc[ind]
    if gend == 0:
        jc_2020['gender'].loc[ind] = 'unspecified'
    elif gend == 1:
        jc_2020['gender'].loc[ind] = 'male'
    elif gend == 2:
        jc_2020['gender'].loc[ind] = 'female'
        
        
jc_2020

C:\Users\deemi\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,tripduration,starttime,stoptime,start_day,start_hour,end_day,end_hour,trip_duration,start_station_id,start_station_name,...,member_type,birth_year,gender,ride_id,rideable_id,rideable_type,member_id,start_day_id,end_day_id,year
0,841,2020-02-01 00:02:13.7650,2020-02-01 00:16:15.4720,Saturday,0,Saturday,0,14.03,3199,Newport Pkwy,...,Subscriber,1980,male,1.0,NaN,NaN,1.0,6.0,6.0,2020
1,206,2020-02-01 00:16:11.3850,2020-02-01 00:19:37.4460,Saturday,0,Saturday,0,3.43,3273,Manila & 1st,...,Subscriber,1990,male,2.0,NaN,NaN,1.0,6.0,6.0,2020
2,859,2020-02-01 00:23:22.1090,2020-02-01 00:37:41.5060,Saturday,0,Saturday,0,14.32,3185,City Hall,...,Customer,1991,male,3.0,NaN,NaN,2.0,6.0,6.0,2020
3,693,2020-02-01 00:25:14.5170,2020-02-01 00:36:48.0280,Saturday,0,Saturday,0,11.57,3185,City Hall,...,Subscriber,1987,male,4.0,NaN,NaN,1.0,6.0,6.0,2020
4,1473,2020-02-01 00:32:30.5050,2020-02-01 00:57:04.3740,Saturday,0,Saturday,0,24.57,3192,Liberty Light Rail,...,Customer,1969,unspecified,5.0,NaN,NaN,2.0,6.0,6.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22957,262,2020-02-29 23:40:53.1620,2020-02-29 23:45:15.3180,Saturday,23,Saturday,23,4.37,3639,Harborside,...,Subscriber,1967,male,22958.0,NaN,NaN,1.0,6.0,6.0,2020
22958,216,2020-02-29 23:41:01.3350,2020-02-29 23:44:38.0350,Saturday,23,Saturday,23,3.62,3186,Grove St PATH,...,Subscriber,1958,male,22959.0,NaN,NaN,1.0,6.0,6.0,2020
22959,101,2020-02-29 23:45:22.1700,2020-02-29 23:47:03.2970,Saturday,23,Saturday,23,1.68,3214,Essex Light Rail,...,Subscriber,1967,male,22960.0,NaN,NaN,1.0,6.0,6.0,2020
22960,279,2020-02-29 23:47:56.8380,2020-02-29 23:52:36.6400,Saturday,23,Saturday,23,4.67,3186,Grove St PATH,...,Subscriber,1988,male,22961.0,NaN,NaN,1.0,6.0,6.0,2020


In [9]:
#Eliminación de columnas sobrantes
col = ['tripduration', 'starttime', 'stoptime', 'bikeid']
for c in col:
    jc_2020.drop([c], axis = 'columns', inplace = True)
    
#Reordenación de las columnas 
jc_2020 = jc_2020.reindex(columns=['ride_id', 'rideable_id', 'rideable_type', 'year',
                                   'start_day_id', 'start_day', 'start_hour', 'end_day_id',
                                   'end_day', 'end_hour', 'trip_duration', 'start_station_id',
                                   'start_station_name', 'start_lat', 'start_lng', 'end_station_id',
                                   'end_station_name', 'end_lat', 'end_lng', 'member_id',
                                   'member_type', 'birth_year', 'gender'])
jc_2020

,ride_id,rideable_id,rideable_type,year,start_day_id,start_day,start_hour,end_day_id,end_day,end_hour,...,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,member_id,member_type,birth_year,gender
0,1.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.728745,-74.032108,3213,Van Vorst Park,40.718489,-74.047727,1.0,Subscriber,1980,male
1,2.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.721651,-74.042884,3639,Harborside,40.719252,-74.034234,1.0,Subscriber,1990,male
2,3.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.717732,-74.043845,3268,Lafayette Park,40.713464,-74.062859,2.0,Customer,1991,male
3,4.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.717732,-74.043845,3277,Communipaw & Berry Lane,40.714358,-74.066611,1.0,Subscriber,1987,male
4,5.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.711242,-74.055701,3639,Harborside,40.719252,-74.034234,2.0,Customer,1969,unspecified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22957,22958.0,NaN,NaN,2020,6.0,Saturday,23,6.0,Saturday,23,...,40.719252,-74.034234,3214,Essex Light Rail,40.712774,-74.036486,1.0,Subscriber,1967,male
22958,22959.0,NaN,NaN,2020,6.0,Saturday,23,6.0,Saturday,23,...,40.719586,-74.043117,3483,Montgomery St,40.719420,-74.050990,1.0,Subscriber,1958,male
22959,22960.0,NaN,NaN,2020,6.0,Saturday,23,6.0,Saturday,23,...,40.712774,-74.036486,3681,Grand St,40.715178,-74.037683,1.0,Subscriber,1967,male
22960,22961.0,NaN,NaN,2020,6.0,Saturday,23,6.0,Saturday,23,...,40.719586,-74.043117,3483,Montgomery St,40.719420,-74.050990,1.0,Subscriber,1988,male


In [10]:
#Y ya se tienen las columnas con el nombre y orden deseados.
[*jc_2020]

['ride_id',
 'rideable_id',
 'rideable_type',
 'year',
 'start_day_id',
 'start_day',
 'start_hour',
 'end_day_id',
 'end_day',
 'end_hour',
 'trip_duration',
 'start_station_id',
 'start_station_name',
 'start_lat',
 'start_lng',
 'end_station_id',
 'end_station_name',
 'end_lat',
 'end_lng',
 'member_id',
 'member_type',
 'birth_year',
 'gender']

## Transformación tabla 2021

In [11]:
jc_2021 = pd.read_csv('citibike//JC-202102-citibike-tripdata.csv')
[*jc_2021]

['ride_id',
 'rideable_type',
 'started_at',
 'ended_at',
 'start_station_name',
 'start_station_id',
 'end_station_name',
 'end_station_id',
 'start_lat',
 'start_lng',
 'end_lat',
 'end_lng',
 'member_casual']

In [12]:
#Se renombran las columnas
jc_2021.rename(columns = {'member_casual': 'member_type'}, inplace = True)

#Nuevas columnas
lista_columnas = transformacion_tiempo(jc_2021, 'started_at', 'ended_at')

col = ['start_day', 'start_hour', 'end_day', 'end_hour', 'trip_duration']

i = 0
for c in range(4,9):
    jc_2021.insert(c, col[i], lista_columnas[i])
    i = i + 1
    
jc_2021

,ride_id,rideable_type,started_at,ended_at,start_day,start_hour,end_day,end_hour,trip_duration,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_type
0,121DD7DD23CB1335,docked_bike,2021-02-03 23:11:28,2021-02-03 23:18:28,Wednesday,23,Wednesday,23,7.00,Hoboken Ave at Monmouth St,JC105,Christ Hospital,JC034,40.735208,-74.046964,40.734786,-74.050444,member
1,FD73FB85F008349D,docked_bike,2021-02-27 16:34:05,2021-02-27 16:56:40,Saturday,16,Saturday,16,22.58,Newport Pkwy,JC008,Marin Light Rail,JC013,40.728744,-74.032108,40.714584,-74.042817,member
2,39F9E6663CB5FDF6,docked_bike,2021-02-26 23:16:04,2021-02-26 23:22:25,Friday,23,Friday,23,6.35,Journal Square,JC103,Baldwin at Montgomery,JC020,40.733670,-74.062500,40.723659,-74.064194,member
3,A64745CB0792EC6F,docked_bike,2021-02-24 16:51:50,2021-02-24 17:16:09,Wednesday,16,Wednesday,17,24.32,Hoboken Ave at Monmouth St,JC105,Hoboken Ave at Monmouth St,JC105,40.735208,-74.046963,40.735208,-74.046964,casual
4,75CC76EB9543764A,docked_bike,2021-02-24 20:44:16,2021-02-24 20:44:46,Wednesday,20,Wednesday,20,0.50,Hoboken Ave at Monmouth St,JC105,Hoboken Ave at Monmouth St,JC105,40.735208,-74.046963,40.735208,-74.046964,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4876,3F8FD9A051C0E3DA,docked_bike,2021-02-24 18:05:45,2021-02-24 18:12:27,Wednesday,18,Wednesday,18,6.70,Columbus Drive,JC014,Newport Pkwy,JC008,40.718355,-74.038914,40.728745,-74.032108,member
4877,B91F3593C0C06D04,docked_bike,2021-02-22 18:22:04,2021-02-22 18:28:45,Monday,18,Monday,18,6.68,Columbus Drive,JC014,Newport Pkwy,JC008,40.718355,-74.038914,40.728745,-74.032108,member
4878,A790954A017D07CB,docked_bike,2021-02-13 12:21:49,2021-02-13 12:27:51,Saturday,12,Saturday,12,6.03,Columbus Drive,JC014,Newport Pkwy,JC008,40.718355,-74.038914,40.728745,-74.032108,member
4879,42B1932D06CEE034,docked_bike,2021-02-25 11:20:29,2021-02-25 11:25:23,Thursday,11,Thursday,11,4.90,Columbus Drive,JC014,Morris Canal,JC072,40.718355,-74.038914,40.712419,-74.038526,member


In [13]:
#Columnas por rellenar
jc_2021['rideable_id'] = pd.Series()
jc_2021['member_id'] = pd.Series()
jc_2021['start_day_id'] = pd.Series()
jc_2021['end_day_id'] = pd.Series()
jc_2021['birth_year'] = pd.Series()
jc_2021['gender'] =pd.Series()
jc_2021['year'] = 2021

<ipython-input-13-734f4e6b3f89>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  jc_2021['rideable_id'] = pd.Series()
<ipython-input-13-734f4e6b3f89>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  jc_2021['member_id'] = pd.Series()
<ipython-input-13-734f4e6b3f89>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  jc_2021['start_day_id'] = pd.Series()
<ipython-input-13-734f4e6b3f89>:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  jc_2021['end_day_id'] = pd.Series()
<ipython-input-13-734f4e6b3f89>:6: Depreca

In [14]:
#Reescritura de la columna 'ride_id'. Al desconocer la nomenclatura de las id que presenta en 2021 y 2022, se decide
# hacer una id enumerada de 1 hasta la cantidad de elementos que sumen las tres tablas.

i = len(jc_2020)  #Se inicia la numeración a partir del último índice + 1 de la tabla previa
for ind in jc_2021.index:
    i = i + 1
    jc_2021['ride_id'].loc[ind] = i
    
    
#Rellenado de la columna 'member_id' considerando la clasificación {1:'member', 2:'casual'}
for ind in jc_2021.index:
    if jc_2021['member_type'].loc[ind] == 'member':
        jc_2021['member_id'].loc[ind] = 1
    else:
        jc_2021['member_id'].loc[ind] = 2
        
#Rellenado de la columna 'start_day_id' y 'end_day_id' considerando la clasificación 
# {1:'monday', 2:'tuesday', 3:'wednesday', 4:'thursday', 5:'friday', 6:'saturday', 7:'sunday'}
dias = {'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4, 'Friday':5, 'Saturday':6, 'Sunday':7}
for ind in jc_2021.index:
    day_start = jc_2021['start_day'].loc[ind]
    jc_2021['start_day_id'].loc[ind] = dias[day_start]
    day_end = jc_2021['end_day'].loc[ind]
    jc_2021['end_day_id'].loc[ind] = dias[day_end]
    
#Rellenado de la columna 'rideable_id' de acuerdo a la clasificación {1: 'classic_bike', 2: 'docked_bike'}
for ind in jc_2021.index:
    if jc_2021['rideable_type'].loc[ind] == 'classic_bike':
        jc_2021['rideable_id'].loc[ind] = 1
    else:
        jc_2021['rideable_id'].loc[ind] = 2
    
jc_2021

C:\Users\deemi\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,ride_id,rideable_type,started_at,ended_at,start_day,start_hour,end_day,end_hour,trip_duration,start_station_name,...,end_lat,end_lng,member_type,rideable_id,member_id,start_day_id,end_day_id,birth_year,gender,year
0,22963,docked_bike,2021-02-03 23:11:28,2021-02-03 23:18:28,Wednesday,23,Wednesday,23,7.00,Hoboken Ave at Monmouth St,...,40.734786,-74.050444,member,2.0,1.0,3.0,3.0,NaN,NaN,2021
1,22964,docked_bike,2021-02-27 16:34:05,2021-02-27 16:56:40,Saturday,16,Saturday,16,22.58,Newport Pkwy,...,40.714584,-74.042817,member,2.0,1.0,6.0,6.0,NaN,NaN,2021
2,22965,docked_bike,2021-02-26 23:16:04,2021-02-26 23:22:25,Friday,23,Friday,23,6.35,Journal Square,...,40.723659,-74.064194,member,2.0,1.0,5.0,5.0,NaN,NaN,2021
3,22966,docked_bike,2021-02-24 16:51:50,2021-02-24 17:16:09,Wednesday,16,Wednesday,17,24.32,Hoboken Ave at Monmouth St,...,40.735208,-74.046964,casual,2.0,2.0,3.0,3.0,NaN,NaN,2021
4,22967,docked_bike,2021-02-24 20:44:16,2021-02-24 20:44:46,Wednesday,20,Wednesday,20,0.50,Hoboken Ave at Monmouth St,...,40.735208,-74.046964,member,2.0,1.0,3.0,3.0,NaN,NaN,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4876,27839,docked_bike,2021-02-24 18:05:45,2021-02-24 18:12:27,Wednesday,18,Wednesday,18,6.70,Columbus Drive,...,40.728745,-74.032108,member,2.0,1.0,3.0,3.0,NaN,NaN,2021
4877,27840,docked_bike,2021-02-22 18:22:04,2021-02-22 18:28:45,Monday,18,Monday,18,6.68,Columbus Drive,...,40.728745,-74.032108,member,2.0,1.0,1.0,1.0,NaN,NaN,2021
4878,27841,docked_bike,2021-02-13 12:21:49,2021-02-13 12:27:51,Saturday,12,Saturday,12,6.03,Columbus Drive,...,40.728745,-74.032108,member,2.0,1.0,6.0,6.0,NaN,NaN,2021
4879,27842,docked_bike,2021-02-25 11:20:29,2021-02-25 11:25:23,Thursday,11,Thursday,11,4.90,Columbus Drive,...,40.712419,-74.038526,member,2.0,1.0,4.0,4.0,NaN,NaN,2021


In [15]:
#Eliminación de columnas sobrantes
col = ['started_at', 'ended_at']
for c in col:
    jc_2021.drop([c], axis = 'columns', inplace = True)
    
#Reordenación de las columnas 
jc_2021 = jc_2021.reindex(columns=['ride_id', 'rideable_id', 'rideable_type', 'year',
                                   'start_day_id', 'start_day', 'start_hour', 'end_day_id',
                                   'end_day', 'end_hour', 'trip_duration', 'start_station_id',
                                   'start_station_name', 'start_lat', 'start_lng', 'end_station_id',
                                   'end_station_name', 'end_lat', 'end_lng', 'member_id',
                                   'member_type', 'birth_year', 'gender'])

jc_2021 = jc_2021.astype({'ride_id':'int64', 'rideable_id':'int64', 'start_day_id':'int64', 
                          'end_day_id':'int64', 'member_id':'int64'})

jc_2021

,ride_id,rideable_id,rideable_type,year,start_day_id,start_day,start_hour,end_day_id,end_day,end_hour,...,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,member_id,member_type,birth_year,gender
0,22963,2,docked_bike,2021,3,Wednesday,23,3,Wednesday,23,...,40.735208,-74.046964,JC034,Christ Hospital,40.734786,-74.050444,1,member,NaN,NaN
1,22964,2,docked_bike,2021,6,Saturday,16,6,Saturday,16,...,40.728744,-74.032108,JC013,Marin Light Rail,40.714584,-74.042817,1,member,NaN,NaN
2,22965,2,docked_bike,2021,5,Friday,23,5,Friday,23,...,40.733670,-74.062500,JC020,Baldwin at Montgomery,40.723659,-74.064194,1,member,NaN,NaN
3,22966,2,docked_bike,2021,3,Wednesday,16,3,Wednesday,17,...,40.735208,-74.046963,JC105,Hoboken Ave at Monmouth St,40.735208,-74.046964,2,casual,NaN,NaN
4,22967,2,docked_bike,2021,3,Wednesday,20,3,Wednesday,20,...,40.735208,-74.046963,JC105,Hoboken Ave at Monmouth St,40.735208,-74.046964,1,member,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4876,27839,2,docked_bike,2021,3,Wednesday,18,3,Wednesday,18,...,40.718355,-74.038914,JC008,Newport Pkwy,40.728745,-74.032108,1,member,NaN,NaN
4877,27840,2,docked_bike,2021,1,Monday,18,1,Monday,18,...,40.718355,-74.038914,JC008,Newport Pkwy,40.728745,-74.032108,1,member,NaN,NaN
4878,27841,2,docked_bike,2021,6,Saturday,12,6,Saturday,12,...,40.718355,-74.038914,JC008,Newport Pkwy,40.728745,-74.032108,1,member,NaN,NaN
4879,27842,2,docked_bike,2021,4,Thursday,11,4,Thursday,11,...,40.718355,-74.038914,JC072,Morris Canal,40.712419,-74.038526,1,member,NaN,NaN


## Transformación tabla 2022
Puesto que los años 2021 y 2022 tienen la misma estructura, basta con replicar casi de forma idéntica el código que se utilizó para la tabla de 2021. Si fueran más años, se podría hacer una función o un ciclo que pase por cada uno de ellos.

In [16]:
jc_2022 = pd.read_csv('citibike//JC-202202-citibike-tripdata.csv')

#Se renombran las columnas
jc_2022.rename(columns = {'member_casual': 'member_type'}, inplace = True)

#Nuevas columnas
lista_columnas = transformacion_tiempo(jc_2022, 'started_at', 'ended_at')

col = ['start_day', 'start_hour', 'end_day', 'end_hour', 'trip_duration']

i = 0
for c in range(4,9):
    jc_2022.insert(c, col[i], lista_columnas[i])
    i = i + 1
    
#Columnas por rellenar
jc_2022['rideable_id'] = pd.Series()
jc_2022['member_id'] = pd.Series()
jc_2022['start_day_id'] = pd.Series()
jc_2022['end_day_id'] = pd.Series()
jc_2022['birth_year'] = pd.Series()
jc_2022['gender'] =pd.Series()
jc_2022['year'] = 2022

#Reescritura de la columna 'ride_id'.
i = len(jc_2020) + len(jc_2021)
for ind in jc_2022.index:
    i = i + 1
    jc_2022['ride_id'].loc[ind] = i
    
    
#Rellenado de la columna 'member_id' considerando la clasificación {1:'member', 2:'casual'}
for ind in jc_2022.index:
    if jc_2022['member_type'].loc[ind] == 'member':
        jc_2022['member_id'].loc[ind] = 1
    else:
        jc_2022['member_id'].loc[ind] = 2
        
#Rellenado de la columna 'start_day_id' y 'end_day_id' considerando la clasificación 
# {1:'monday', 2:'tuesday', 3:'wednesday', 4:'thursday', 5:'friday', 6:'saturday', 7:'sunday'}
dias = {'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4, 'Friday':5, 'Saturday':6, 'Sunday':7}
for ind in jc_2022.index:
    day_start = jc_2022['start_day'].loc[ind]
    jc_2022['start_day_id'].loc[ind] = dias[day_start]
    day_end = jc_2022['end_day'].loc[ind]
    jc_2022['end_day_id'].loc[ind] = dias[day_end]
    
#Rellenado de la columna 'rideable_id' de acuerdo a la clasificación {1: 'classic_bike', 2: 'docked_bike'}
for ind in jc_2022.index:
    if jc_2022['rideable_type'].loc[ind] == 'classic_bike':
        jc_2022['rideable_id'].loc[ind] = 1
    else:
        jc_2022['rideable_id'].loc[ind] = 2
        
#Eliminación de columnas sobrantes
col = ['started_at', 'ended_at']
for c in col:
    jc_2022.drop([c], axis = 'columns', inplace = True)
    
#Reordenación de las columnas 
jc_2022 = jc_2022.reindex(columns=['ride_id', 'rideable_id', 'rideable_type', 'year',
                                   'start_day_id', 'start_day', 'start_hour', 'end_day_id',
                                   'end_day', 'end_hour', 'trip_duration', 'start_station_id',
                                   'start_station_name', 'start_lat', 'start_lng', 'end_station_id',
                                   'end_station_name', 'end_lat', 'end_lng', 'member_id',
                                   'member_type', 'birth_year', 'gender'])

jc_2022 = jc_2022.astype({'ride_id':'int64', 'rideable_id':'int64', 'start_day_id':'int64', 
                          'end_day_id':'int64', 'member_id':'int64'})

jc_2022

<ipython-input-16-b355f3e0d1d1>:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  jc_2022['rideable_id'] = pd.Series()
<ipython-input-16-b355f3e0d1d1>:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  jc_2022['member_id'] = pd.Series()
<ipython-input-16-b355f3e0d1d1>:19: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  jc_2022['start_day_id'] = pd.Series()
<ipython-input-16-b355f3e0d1d1>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  jc_2022['end_day_id'] = pd.Series()
<ipython-input-16-b355f3e0d1d1>:21: De

,ride_id,rideable_id,rideable_type,year,start_day_id,start_day,start_hour,end_day_id,end_day,end_hour,...,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,member_id,member_type,birth_year,gender
0,27844,1,classic_bike,2022,1,Monday,17,1,Monday,17,...,40.734786,-74.050444,HB601,Church Sq Park - 5 St & Park Ave,40.742659,-74.032233,1,member,NaN,NaN
1,27845,1,classic_bike,2022,3,Wednesday,8,3,Wednesday,8,...,40.727551,-74.071061,JC006,Warren St,40.721124,-74.038051,1,member,NaN,NaN
2,27846,1,classic_bike,2022,1,Monday,17,1,Monday,17,...,40.734786,-74.050444,JC009,Hamilton Park,40.727596,-74.044247,1,member,NaN,NaN
3,27847,1,classic_bike,2022,7,Sunday,16,7,Sunday,16,...,40.748161,-74.032453,HB105,City Hall - Washington St & 1 St,40.737360,-74.030970,2,casual,NaN,NaN
4,27848,1,classic_bike,2022,6,Saturday,14,6,Saturday,14,...,40.737430,-74.035710,JC018,5 Corners Library,40.734961,-74.059503,2,casual,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31906,59750,1,classic_bike,2022,7,Sunday,12,7,Sunday,12,...,40.721124,-74.038051,JC078,Lafayette Park,40.713464,-74.062859,1,member,NaN,NaN
31907,59751,1,classic_bike,2022,4,Thursday,20,4,Thursday,20,...,40.719586,-74.043117,JC078,Lafayette Park,40.713464,-74.062859,1,member,NaN,NaN
31908,59752,1,classic_bike,2022,3,Wednesday,19,3,Wednesday,19,...,40.719586,-74.043117,JC078,Lafayette Park,40.713464,-74.062859,1,member,NaN,NaN
31909,59753,1,classic_bike,2022,3,Wednesday,18,3,Wednesday,18,...,40.719586,-74.043117,JC009,Hamilton Park,40.727596,-74.044247,1,member,NaN,NaN


## Apilación de las tres tablas

In [17]:
registros = pd.DataFrame()

registros = registros.append(jc_2020)
registros = registros.append(jc_2021)
registros = registros.append(jc_2022)

registros

,ride_id,rideable_id,rideable_type,year,start_day_id,start_day,start_hour,end_day_id,end_day,end_hour,...,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,member_id,member_type,birth_year,gender
0,1.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.728745,-74.032108,3213,Van Vorst Park,40.718489,-74.047727,1.0,Subscriber,1980.0,male
1,2.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.721651,-74.042884,3639,Harborside,40.719252,-74.034234,1.0,Subscriber,1990.0,male
2,3.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.717732,-74.043845,3268,Lafayette Park,40.713464,-74.062859,2.0,Customer,1991.0,male
3,4.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.717732,-74.043845,3277,Communipaw & Berry Lane,40.714358,-74.066611,1.0,Subscriber,1987.0,male
4,5.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.711242,-74.055701,3639,Harborside,40.719252,-74.034234,2.0,Customer,1969.0,unspecified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31906,59750.0,1.0,classic_bike,2022,7.0,Sunday,12,7.0,Sunday,12,...,40.721124,-74.038051,JC078,Lafayette Park,40.713464,-74.062859,1.0,member,NaN,NaN
31907,59751.0,1.0,classic_bike,2022,4.0,Thursday,20,4.0,Thursday,20,...,40.719586,-74.043117,JC078,Lafayette Park,40.713464,-74.062859,1.0,member,NaN,NaN
31908,59752.0,1.0,classic_bike,2022,3.0,Wednesday,19,3.0,Wednesday,19,...,40.719586,-74.043117,JC078,Lafayette Park,40.713464,-74.062859,1.0,member,NaN,NaN
31909,59753.0,1.0,classic_bike,2022,3.0,Wednesday,18,3.0,Wednesday,18,...,40.719586,-74.043117,JC009,Hamilton Park,40.727596,-74.044247,1.0,member,NaN,NaN


## Última transformación: Homologación de los id de las estaciones
Se condisera que las estaciones son inamovibles, por lo que los elementos unicos van a estar dados por el par de coordenadas. A partir de esos pares únicos, se genera un diccionario de identificadores y nombres de estaciones.

In [18]:
#Se reajustan los índices del DataFrame
nuevos_indices = list(range(len(registros)))
registros = registros.set_index(pd.Index(nuevos_indices))

In [19]:
#Paquetería para trabajar con los nan
import math

In [20]:
#Registro de las coordenadas de las estaciones de acuerdo a sus nombres, similar 
# a como estarán en la tabla en el modelo relacional. 
lista_estaciones = list()
nombres_aux = list()
nombre_id = dict()

i = 0
for ind in registros.index:
    
    nombre_ini = registros['start_station_name'].loc[ind]
    nombre_fin = registros['end_station_name'].loc[ind]
    
    if str(nombre_fin) == 'nan':
        continue
    
    if nombre_ini not in nombres_aux:
        lista_aux = list()
        x_start = registros['start_lng'].loc[ind]
        y_start = registros['start_lat'].loc[ind]
        nombres_aux.append(nombre_ini)
        
        i = i + 1
        lista_aux.append(i)
        lista_aux.append(nombre_ini)
        lista_aux.append(round(x_start,3))
        lista_aux.append(round(y_start,3))
        lista_estaciones.append(lista_aux)
        nombre_id[nombre_ini] = i
        
    elif nombre_fin not in nombres_aux:
        x_end = registros['end_lng'].loc[ind]
        y_end = registros['end_lat'].loc[ind]
        if math.isnan(x_end) == True or math.isnan(y_end) == True:
            continue
        else:
            lista_aux = list()
            nombres_aux.append(nombre_fin)
            
            i = i + 1
            lista_aux.append(i)
            lista_aux.append(nombre_fin)
            lista_aux.append(round(x_end,3))
            lista_aux.append(round(y_end,3))
            lista_estaciones.append(lista_aux)
            nombre_id[nombre_fin] = i

In [21]:
#Con estas nuevas id, se sobreescribe en la columna de 'start_station_id' y 'end_station_id'

for ind in registros.index:
    nombre_ini = registros['start_station_name'].loc[ind]
    registros['start_station_id'].loc[ind] = nombre_id[nombre_ini]
    nombre_fin = registros['end_station_name'].loc[ind]
    if str(nombre_fin) == 'nan':
        continue
    else:
        registros['end_station_id'].loc[ind] = nombre_id[nombre_fin]

registros

C:\Users\deemi\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,ride_id,rideable_id,rideable_type,year,start_day_id,start_day,start_hour,end_day_id,end_day,end_hour,...,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,member_id,member_type,birth_year,gender
0,1.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.728745,-74.032108,10,Van Vorst Park,40.718489,-74.047727,1.0,Subscriber,1980.0,male
1,2.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.721651,-74.042884,13,Harborside,40.719252,-74.034234,1.0,Subscriber,1990.0,male
2,3.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.717732,-74.043845,12,Lafayette Park,40.713464,-74.062859,2.0,Customer,1991.0,male
3,4.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.717732,-74.043845,4,Communipaw & Berry Lane,40.714358,-74.066611,1.0,Subscriber,1987.0,male
4,5.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.711242,-74.055701,13,Harborside,40.719252,-74.034234,2.0,Customer,1969.0,unspecified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59749,59750.0,1.0,classic_bike,2022,7.0,Sunday,12,7.0,Sunday,12,...,40.721124,-74.038051,12,Lafayette Park,40.713464,-74.062859,1.0,member,NaN,NaN
59750,59751.0,1.0,classic_bike,2022,4.0,Thursday,20,4.0,Thursday,20,...,40.719586,-74.043117,12,Lafayette Park,40.713464,-74.062859,1.0,member,NaN,NaN
59751,59752.0,1.0,classic_bike,2022,3.0,Wednesday,19,3.0,Wednesday,19,...,40.719586,-74.043117,12,Lafayette Park,40.713464,-74.062859,1.0,member,NaN,NaN
59752,59753.0,1.0,classic_bike,2022,3.0,Wednesday,18,3.0,Wednesday,18,...,40.719586,-74.043117,9,Hamilton Park,40.727596,-74.044247,1.0,member,NaN,NaN


In [22]:
edad = list(registros['year'] - registros['birth_year'])
registros.insert(21, 'age', edad)

registros.drop(['birth_year'], axis = 'columns', inplace = True)

registros

,ride_id,rideable_id,rideable_type,year,start_day_id,start_day,start_hour,end_day_id,end_day,end_hour,...,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,member_id,member_type,age,gender
0,1.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.728745,-74.032108,10,Van Vorst Park,40.718489,-74.047727,1.0,Subscriber,40.0,male
1,2.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.721651,-74.042884,13,Harborside,40.719252,-74.034234,1.0,Subscriber,30.0,male
2,3.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.717732,-74.043845,12,Lafayette Park,40.713464,-74.062859,2.0,Customer,29.0,male
3,4.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.717732,-74.043845,4,Communipaw & Berry Lane,40.714358,-74.066611,1.0,Subscriber,33.0,male
4,5.0,NaN,NaN,2020,6.0,Saturday,0,6.0,Saturday,0,...,40.711242,-74.055701,13,Harborside,40.719252,-74.034234,2.0,Customer,51.0,unspecified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59749,59750.0,1.0,classic_bike,2022,7.0,Sunday,12,7.0,Sunday,12,...,40.721124,-74.038051,12,Lafayette Park,40.713464,-74.062859,1.0,member,NaN,NaN
59750,59751.0,1.0,classic_bike,2022,4.0,Thursday,20,4.0,Thursday,20,...,40.719586,-74.043117,12,Lafayette Park,40.713464,-74.062859,1.0,member,NaN,NaN
59751,59752.0,1.0,classic_bike,2022,3.0,Wednesday,19,3.0,Wednesday,19,...,40.719586,-74.043117,12,Lafayette Park,40.713464,-74.062859,1.0,member,NaN,NaN
59752,59753.0,1.0,classic_bike,2022,3.0,Wednesday,18,3.0,Wednesday,18,...,40.719586,-74.043117,9,Hamilton Park,40.727596,-74.044247,1.0,member,NaN,NaN


In [23]:
registros.to_csv('total_registros.csv', index = False)